In [181]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [182]:
N = 5  # 5 periods: 0-0.25, 0.25-0.5, 0.5-0.75, 0.75-1, 1-1.25
dt = 0.25
r0_trs = 0.025
n_path = 10000

In [183]:
# treasury params
alpha = 0.05
sigma_trs = 0.1
theta_t = np.array([0.028, 0.3, 0.4, 0.5, 0.6])

# ted and Libor params
a = 0.5
b = 0.04
sigma_ted = 0.1
rho = 0.2
r0_libor = 0.0362

## Simulation given model parameters

In [184]:
def generate_r_trs_helper(dW, alpha, sigma_trs, theta_t, n_path, N, dt, r0_trs):
    r = np.zeros((n_path, N))
    r[:, 0] = r0_trs
    for i in range(1, N):
        r[:, i] = r[:,i-1] + alpha*(theta_t[i-1]-r[:,i-1])*dt + sigma_trs*dW[:,i-1]
    return r

In [185]:
def generate_r_ted_helper(dW, a, b, sigma_ted, rho, r0_libor, n_path, N, dt, r0_trs):
    ted = np.zeros((n_path, N))
    ted[:, 0] = r0_libor - r0_trs
    for i in range(1, N):
        ted[:, i] = (ted[:,i-1] + a*(b-ted[:,i-1])*dt 
                     + sigma_ted*np.sqrt(ted[:,i-1])*dW[:,i-1] 
                     + 0.25*(sigma_ted**2)*(dW[:,i-1]**2-dt))
        ted[:, i] = np.abs(ted[:, i])  # reflecting fix
    return ted

In [186]:
def generate_funding_spread_helper(dW, n_path, N, dt):
    x = np.zeros((n_path, N))
    x[:, 0] = 9e-4
    for i in range(1, N):
        x[:, i] = x[:,i-1] + 3*(0.001-x[:,i-1])*dt + 0.15*x[:,i-1]*dW[:,i-1]
    return x

In [187]:
def generate_interest_rates(alpha, sigma_trs, theta_t, a, b, sigma_ted, rho, r0_libor,
                            n_path=10000, N=5, dt=0.25, r0_trs=0.025):
    '''
    Generate treasury spot rate, ted spread, libor spot rate, funding rate
    treasury parameters:
        alpha, sigma_trs: scalar
        theta_t: np.array, len=N
    ted spread and Libor parameters:
        a, b, sigma_ted, rho, r0_libor: scalar
    '''
    # generate r_treasury paths using Euler Scheme and antithetic variable
    dW_trs = np.random.normal(0, np.sqrt(dt), size=(n_path, N))
    r_trs1 = generate_r_trs_helper(dW_trs, alpha, sigma_trs, theta_t, n_path, N, dt, r0_trs)
    r_trs2 = generate_r_trs_helper(-dW_trs, alpha, sigma_trs, theta_t, n_path, N, dt, r0_trs)  # antithetic
    r_trs = np.concatenate([r_trs1, r_trs2])
    
    # generate ted spread using Milstein Scheme, and Libor (r_treasury + ted spread) and their antithetic paths
    dW_tmp = np.random.normal(0, np.sqrt(dt), size=(n_path, N))
    dW_ted = rho*dW_trs + np.sqrt(1-rho**2)*dW_tmp
    r_ted1 = generate_r_ted_helper(dW_ted, a, b, sigma_ted, rho, r0_libor, n_path, N, dt, r0_trs)
    r_ted2 = generate_r_ted_helper(-dW_ted, a, b, sigma_ted, rho, r0_libor, n_path, N, dt, r0_trs)  # antithetic
    r_ted = np.concatenate([r_ted1, r_ted2])
    r_libor = r_trs + r_ted
    
    # generate funding rate
    dW_x = np.random.normal(0, np.sqrt(dt), size=(n_path, N))
    x1 = generate_funding_spread_helper(dW_x, n_path, N, dt)
    x2 = generate_funding_spread_helper(-dW_x, n_path, N, dt)  # antithetic
    x = np.concatenate([x1, x2])
    r_funding = r_libor + x
    return r_trs, r_ted, r_libor, r_funding

In [188]:
r_trs, r_ted, r_libor, r_funding = generate_interest_rates(alpha, sigma_trs, theta_t,
                                                           a, b, sigma_ted, rho, r0_libor,
                                                           n_path=10000, N=5, dt=0.25, r0_trs=0.025)

## Calibration

Goal: find the reasonable parameters a, b, sigma_ted to minimize the loss between true market price and simulated market price.

Several constraints:

- 2ab $\geq$ sigma_ted^2, to ensure that ted is a non-negative process
- Simulated market price must coincide with the strucutre of true market price: both Libor rate and caplet price should be increasing

In [189]:
# true market libor rate and caplet price
mkt_libor = np.array([0.0362, 0.0363, 0.0371, 0.0372, 0.0379])
mkt_libor_cap = np.array([np.nan, 0.00081, 0.00108, 0.00128, 0.00133])
tmp = (1+dt*mkt_libor) ** np.arange(1, N+1)
mkt_forward = (tmp / np.insert(tmp[:-1], 0, 1) - 1) / dt

In [190]:
# simulated market libor rate and caplet price
sim_libor = (np.cumprod(1+dt*r_libor, axis=1) ** (1/np.arange(1, N+1)) - 1) / dt
sim_libor = np.mean(sim_libor, axis=0)
cap_payoff = np.maximum(r_libor - mkt_forward, np.zeros(r_libor.shape))
pv = 1 / np.cumprod(1+dt*r_trs, axis=1)  # discount by treasury rate
sim_libor_cap = np.mean(pv * cap_payoff, axis=0)
sim_libor_cap[0] = np.nan

In [196]:
print('true market libor:\n', mkt_libor)
print('simulated market libor:\n', sim_libor)
print('\n')
print('true market libor caplet price:\n', mkt_libor_cap)
print('simulated market libor caplet price:\n', sim_libor_cap)

true market libor:
 [0.0362 0.0363 0.0371 0.0372 0.0379]
simulated market libor:
 [0.0362     0.03793651 0.04067457 0.04385917 0.04739302]


true market libor caplet price:
 [    nan 0.00081 0.00108 0.00128 0.00133]
simulated market libor caplet price:
 [       nan 0.0214993  0.03125902 0.0407793  0.04665703]
